In [1]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
BASE_IMG_DIR="../../data/raw/03_AIHUB_dataset/images"
BASE_JSON_DIR="../../data/raw/03_AIHUB_dataset/labeling"
SAVE_DIR="../../data/processed/03_AIHUB_dataset/cropped_data"

folder="001_G1"
filename="001_G1_01_무광원_계기판_정상주시_20200917_182234_02144.jpg"
img_path=os.path.join(BASE_IMG_DIR, folder, filename)
json_path=os.path.join(BASE_JSON_DIR, folder, filename.replace(".jpg", ".json"))

In [3]:
img=cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

with open(json_path, "r") as f:
    data=json.load(f)
# print(data)

In [4]:
leye_box=data["ObjectInfo"]["BoundingBox"]["Leye"]["Position"]
reye_box=data["ObjectInfo"]["BoundingBox"]["Reye"]["Position"]

def crop_eye(img, bbox):
    x1, y1, x2, y2 = bbox
    return img[y1:y2, x1:x2]

leye_img=crop_eye(img, leye_box)
reye_img=crop_eye(img, reye_box)

print("Left eye shape:", leye_img.shape)
print("Right eye shape:", reye_img.shape)


# 개인정보 이므로 출력은 지우기

# plt.figure(figsize=(10, 4))

# plt.subplot(1, 2, 1)
# plt.title("Left Eye")
# plt.imshow(leye_img, cmap="gray")

# plt.subplot(1, 2, 2)
# plt.title("Right Eye")
# plt.imshow(reye_img, cmap="gray")

# plt.show()

Left eye shape: (41, 82)
Right eye shape: (41, 81)


# 전체 이미지 전처리

In [5]:
IMG_SIZE=(80, 40)

In [6]:
# 파일명에 따라 open/closed 라벨 반환
def get_label_from_filename(filename):
    if "정상주시" in filename:
        return "open"
    elif "졸음재현" in filename or "하품재현" in filename:
        return "closed"
    else:
        return None

In [9]:
import glob

def process_all_with_glob():
    img_paths=glob.glob(os.path.join(BASE_IMG_DIR, "*", "*.jpg"))

    for img_path in tqdm(img_paths, desc="Processing images", ncols=100):
        filename=os.path.basename(img_path)
        folder=os.path.basename(os.path.dirname(img_path))
        json_path=os.path.join(BASE_JSON_DIR, folder, filename.replace(".jpg", ".json"))

        label=get_label_from_filename(filename)
        if label is None or not os.path.exists(json_path): continue

        img=cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None: continue

        with open(json_path, "r") as f:
            data=json.load(f)

        try:
            leye_box=data["ObjectInfo"]["BoundingBox"]["Leye"]["Position"]
            reye_box=data["ObjectInfo"]["BoundingBox"]["Reye"]["Position"]

            for LorR, bbox in [("left", leye_box), ("right", reye_box)]:
                x1, y1, x2, y2=bbox
                eye_crop=img[y1:y2, x1:x2]
                
                if eye_crop is None or eye_crop.size==0: continue
                
                eye_resized=cv2.resize(eye_crop, IMG_SIZE)  # crop한 눈 resize -> 80, 40

                save_dir=os.path.join(SAVE_DIR, label, LorR)
                os.makedirs(save_dir, exist_ok=True)
                save_path=os.path.join(save_dir, filename)
                
                cv2.imwrite(save_path, eye_resized)

        except KeyError:
            continue

In [10]:
process_all_with_glob()

Processing images: 100%|███████████████████████████████████| 112866/112866 [03:30<00:00, 536.73it/s]


In [17]:
for label in ["open", "closed"]:
    for LorR in ["left", "right"]:
        path=os.path.join(SAVE_DIR, label, LorR)
        print(f"{label}: {LorR} -> {len(os.listdir(path))}개")

open: left -> 48820개
open: right -> 46393개
closed: left -> 36365개
closed: right -> 33184개


# sample 4개 찍어보기

In [ ]:
# import random

# categories=[("closed", "left"),
#             ("closed", "right"),
#             ("open", "left"),
#             ("open", "right")]

# plt.figure(figsize=(8, 6))

# for i, (label, LorR) in enumerate(categories):
#     path=os.path.join(SAVE_DIR, label, LorR)
#     files=os.listdir(path)
#     if not files:
#         print(f"{path}에 이미지가 없습니다.")
#         continue
#     sample_img=cv2.imread(os.path.join(path, random.choice(files)), cv2.IMREAD_GRAYSCALE)

#     plt.subplot(2, 2, i+1)
#     plt.imshow(sample_img, cmap="gray")
#     plt.title(f"{label}: {LorR}")
#     plt.axis("off")

# plt.tight_layout()
# plt.show()

# .npy로 저장
- 전체 눈 이미지 load 반복하는 건 속도가 느림 -> .npy로 파일로 저장  
- X(이미지), Y(label)로 구성  
- GrayScale 이미지니까 shape는 (h, w, 1)로  
- 모든 이미지를 numpy 배열로 만들어 X.npy, Y.npy로 저장  

In [21]:
import numpy as np

IMG_SIZE=(80, 40)  # (width, height)
base_dir="../../data/processed/03_AIHUB_dataset/cropped_data"

X=[]
Y=[]

for label in ["open", "closed"]:  
    y=1 if label == "open" else 0  # open=1, closed=0

    for LorR in ["left", "right"]:
        folder=os.path.join(base_dir, label, LorR)
        files=os.listdir(folder)
        for file in tqdm(files, desc=f"{label}: {LorR}", ncols=100):
            file_path=os.path.join(folder, file)
            img=cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # GrayScaling
            if img is None: continue
            
            # (40, 80) -> (40, 80, 1)
            img = np.expand_dims(img, axis=-1)
            X.append(img)
            Y.append(y)
X=np.array(X)
Y=np.array(Y)
print(f"X: {X.shape} / Y: {Y.shape}")

closed: right: 100%|████████████████████████████████████████| 33184/33184 [00:07<00:00, 4181.96it/s]


X: (164762, 40, 80, 1) / Y: (164762,)


In [22]:
np.save("../../data/processed/03_AIHUB_dataset/X_eye.npy", X)
np.save("../../data/processed/03_AIHUB_dataset/y_eye.npy", Y)

In [ ]:
# test
X=np.load("../../data/processed/03_AIHUB_dataset/X_eye.npy")
Y=np.load("../../data/processed/03_AIHUB_dataset/y_eye.npy")

print(f"X: {X.shape} / Y: {Y.shape}")
print("Class 분포:", np.unique(y, return_counts=True))

X: (164762, 40, 80, 1) / Y: (164762,)
Class 분포: (array([0, 1]), array([69549, 95213]))
